In [5]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
sys.path.insert(0, PROJECT_ROOT)


In [6]:
import cv2
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from datetime import datetime
from tqdm import tqdm

from src.svm_detector.features.hog import extract_hog
from src.svm_detector.inference.sliding_window import run_svm_inference
from src.svm_detector.inference.nms import nms
from config import config
from pathlib import Path

In [11]:
# ---------------- CONFIG ----------------
PROJECT_ROOT = Path.cwd().parent
WINDOW_SIZE = config.WIN_SIZE
STEP = 16
# THRESHOLD = 0.5
IS_NMS = True

IMAGE_DIR = f"{PROJECT_ROOT}/dataset/yolo/test/images"
LABEL_DIR = f"{PROJECT_ROOT}/dataset/yolo/test/labels"

SVM_PATH = f"{PROJECT_ROOT}/models/svm/svm_chicken_v1.5_base.pkl"

In [8]:
def apply_nms(detections, score_thresh=0.0, nms_thresh=0.4):
    if len(detections) == 0:
        return []

    boxes = []
    scores = []

    for det in detections:
        x1, y1, x2, y2 = det[:4]
        boxes.append([x1, y1, x2 - x1, y2 - y1])
        scores.append(float(det[4]))

    indices = nms(boxes, scores, score_thresh, nms_thresh)
    if len(indices) == 0:
        return []

    return [detections[i] for i in indices.flatten()]

In [9]:
def count_gt_from_yolo(label_path):
    if not os.path.exists(label_path):
        return 0
    with open(label_path, "r") as f:
        return len(f.readlines())

In [13]:
svm = joblib.load(SVM_PATH)

gt_counts = []
pred_counts = []

image_files = [
    f for f in os.listdir(IMAGE_DIR)
    if f.endswith((".jpg", ".png", ".jpeg"))
]

columns = [
    "score_threshold",
    "nms_threshold",
    "mae",
    "rmse",
]

df = pd.DataFrame(columns=columns)

thresholds = np.arange(0.2, 0.9, 0.1)
nms_thresholds = np.arange(0.2, 0.9, 0.1)

for nms_thresh in nms_thresholds:
    for THRESHOLD in thresholds:
        for img_name in tqdm(
            image_files,
            desc="Evaluating counting",
            unit="image"
        ):
            img_path = os.path.join(IMAGE_DIR, img_name)
            label_path = os.path.join(
                LABEL_DIR,
                os.path.splitext(img_name)[0] + ".txt"
            )

            img = cv2.imread(img_path)
            if img is None:
                continue

            # --- inference ---
            detections = run_svm_inference(
                img=img,
                svm=svm,
                feature_extractor=extract_hog,
                window_size=WINDOW_SIZE,
                step=STEP,
                threshold=THRESHOLD,
            )

            if IS_NMS:
                detections = apply_nms(
                    detections,
                    score_thresh=THRESHOLD,
                    nms_thresh=nms_thresh,
                )

            # --- counting ---
            pred_count = len(detections)
            gt_count = count_gt_from_yolo(label_path)

            pred_counts.append(pred_count)
            gt_counts.append(gt_count) 

        # ---------------- METRICS ----------------
        mae = mean_absolute_error(gt_counts, pred_counts)
        rmse = np.sqrt(((np.array(gt_counts) - np.array(pred_counts)) ** 2).mean())

        print("===================================")
        print(f"Test images     : {len(gt_counts)}")
        print(f"Counting MAE    : {mae:.4f}")
        print(f"Counting RMSE   : {rmse:.4f}")
        print(f"Score Threshold   : {THRESHOLD:.4f}")
        print(f"Nms Threshold   : {nms_thresh:.4f}")
        print("===================================") 
        
        row = {"score_threshold": THRESHOLD,
                "nms_threshold": nms_thresh,
                "mae": mae,
                "rmse": rmse,}
        df.loc[len(df)] = row

Evaluating counting: 100%|██████████| 1000/1000 [01:50<00:00,  9.05image/s]


Test images     : 1000
Counting MAE    : 17.8190
Counting RMSE   : 24.9213
Score Threshold   : 0.2000
Nms Threshold   : 0.2000


Evaluating counting: 100%|██████████| 1000/1000 [01:51<00:00,  8.94image/s]


Test images     : 2000
Counting MAE    : 15.7750
Counting RMSE   : 23.5644
Score Threshold   : 0.3000
Nms Threshold   : 0.2000


Evaluating counting: 100%|██████████| 1000/1000 [01:53<00:00,  8.80image/s]


...

Test images     : 47000
Counting MAE    : 11.3864
Counting RMSE   : 21.7170
Score Threshold   : 0.6000
Nms Threshold   : 0.8000


Evaluating counting: 100%|██████████| 1000/1000 [01:52<00:00,  8.87image/s]


Test images     : 48000
Counting MAE    : 11.2930
Counting RMSE   : 21.6614
Score Threshold   : 0.7000
Nms Threshold   : 0.8000


Evaluating counting: 100%|██████████| 1000/1000 [03:42<00:00,  4.49image/s]

Test images     : 49000
Counting MAE    : 11.1967
Counting RMSE   : 21.6094
Score Threshold   : 0.8000
Nms Threshold   : 0.8000


In [14]:
df.to_csv("mea.csv", index=False)

In [16]:
df.loc[df["mae"].idxmin()]

score_threshold     0.800000
nms_threshold       0.200000
mae                10.232857
rmse               20.562316
Name: 6, dtype: float64